In [1]:
import sys
import os

SCRIPT_PATH = os.path.join(os.path.abspath(os.path.dirname('../__file__')), 'script')
sys.path.insert(0, SCRIPT_PATH)


from markov_pyactup import *
from statsmodels.formula.api import logit, glm
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)


main_dir = os.path.dirname(os.getcwd())
subject_dir = os.path.join(main_dir, 'data', 'human', '"online_data')
pr_dir = os.path.join(main_dir, 'data', 'model', 'param_recovery')
opt_dir = os.path.join(main_dir, 'data', 'model', 'param_optimization_0403')

main_dir, subject_dir, opt_dir

sns.set_theme(style="white", rc={"axes.spines.right": True, "axes.spines.top": True}, font_scale=1.3)

Error trying to connect to ACT-R at 127.0.0.1 : 2651 with exception (<class 'ConnectionRefusedError'>, ConnectionRefusedError(111, 'Connection refused'), <traceback object at 0x7f4becafe900>)
ACT-R connection NOT established, but no exception detected or already handled.
Could not start ACT-R connection.


## Run Parameter Recovery 

In [4]:
subject_ids = [1,2]
dfo = MarkovEstimation.load_optimization_data(opt_dir=opt_dir,
                                      estimate_models=['markov-rl-hybrid', 'markov-ibl-hybrid'],
                                      long_format=False,
                                      only_maxLL=True)
dfo[dfo['subject_id'].isin(subject_ids)]


,alpha,beta,lambda_parameter,p_parameter,w_parameter,temperature,decay,lf,fixed_cost,maxLL,estimate_model,subject_id
0,1.0000,1.0047,0.7697,0.0,0.2887,0.2000,0.5000,0.5,0.0,-264.9156,markov-rl-hybrid,1
1,0.9320,2.2464,0.3731,0.0,0.3629,1.8174,1.3051,0.5,0.0,-268.9632,markov-ibl-hybrid,1
2,0.9459,0.9394,0.5169,0.0,0.0000,0.2000,0.5000,0.5,0.0,-267.8429,markov-rl-hybrid,2
3,0.5590,1.4825,0.5180,0.0,0.1036,1.0372,0.9914,0.5,0.0,-270.1964,markov-ibl-hybrid,2


In [2]:
MarkovSimulation.simulate_param_recovery(pr_dir=pr_dir, load_opt=True, verbose=True, overwrite=False)

...SKIP ORI OPT...
...SKIP SINGLE RUN SIM...
START OPT REC...



KeyboardInterrupt



## Correlation

In [ ]:
df_ori = pd.concat([pd.read_csv(f) for f in glob.glob(os.path.join(pr_dir, '*ori*', '*'))], axis=0).melt(id_vars=['subject_id', 'estimate_model'], var_name='param_name', value_name='param_value').sort_values(by = ['subject_id', 'estimate_model'])
df_rec = pd.concat([pd.read_csv(f) for f in glob.glob(os.path.join(pr_dir, '*rec*', '*'))], axis=0).melt(id_vars=['subject_id', 'estimate_model'], var_name='param_name', value_name='param_value').sort_values(by = ['subject_id', 'estimate_model'])

In [ ]:
dfm1 = pd.merge(df_ori[df_ori['estimate_model']=='markov-rl-hybrid'], df_rec[df_rec['estimate_model']=='markov-rl-hybrid'], 
         on=['subject_id', 'param_name', 'estimate_model'], suffixes=('.ori', '.rec')).drop_duplicates()
dfm2 = pd.merge(df_ori[df_ori['estimate_model']=='markov-ibl-hybrid'], df_rec[df_rec['estimate_model']=='markov-ibl-hybrid'],
         on=['subject_id', 'param_name', 'estimate_model'], suffixes=('.ori', '.rec'))

In [ ]:
MarkovPlot.parameter_lm_plot(df=dfm1, x_name='param_value.ori', y_name='param_value.rec', exclude_parameters=['temperature', 'decay', 'lf', 'fixed_cost'], alpha=.1)

In [ ]:
MarkovPlot.parameter_lm_plot(df=dfm2, x_name='param_value.ori', y_name='param_value.rec', exclude_parameters=['beta', 'lf', 'fixed_cost'], alpha=.1)